In [1]:
import torch
import transformers
from tqdm import tqdm
import torch.nn as nn
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import gc
import json
from time import time
import os
import albumentations as A
import cv2
from dataclasses import dataclass
import torch.nn.functional as F

import sys
sys.path.insert(0, "/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/")

from src.readers.fid import FiDReader

In [2]:
reader = FiDReader()

/home/dzigen/miniconda3/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you.

In [3]:
input_ids = torch.randint(0, 255, size=(2,2,512))
attention_mask = torch.randint(0, 255, size=(2,2,512))
labels = torch.randint(0, 255, size=(2,512))

In [ ]:
out = reader.model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=10)

In [11]:
reader.tokenizer.batch_decode(out, skip_special_tokens=True)

['', '']

In [5]:
out.shape

torch.Size([2, 10])

In [4]:
out = reader.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

forward 2
init dimension:  torch.Size([2, 2, 512])
2d resize:  torch.Size([2, 1024])
forward 3
2d resize:  torch.Size([2, 1024])
---
candidates flat:  torch.Size([4, 512])
encoder output  torch.Size([4, 512, 768])
candidates concatenation:
last_hidden_state torch.Size([2, 1024, 768])


### Only Reader Train

### Only Retriever Train

In [86]:
criterion = nn.CrossEntropyLoss()

In [92]:
criterion = nn.CrossEntropyLoss()

output_score = torch.eye(6) #torch.randn(6,6)
targets = torch.arange(0,6)

auto_loss = criterion(output_score, targets)
print(auto_loss)

manual_loss = torch.mean(-torch.log(F.softmax(output_score, dim=1).gather(1, targets.view(-1,1)) ))
print(manual_loss)

tensor(1.0436)
tensor(1.0436)


In [84]:
reader_topk_loss = 0.2
reader_k_loss = torch.tensor([[0.9,0.1,0.5]])
retriever_k_scores = torch.tensor([[5,1,3]])
criterion(reader_topk_loss, reader_k_loss, retriever_k_scores)

tensor(0.0261)

In [83]:
reader_topk_loss = 0.8
reader_k_loss = torch.tensor([[0.9,0.8,1]])
retriever_k_scores = torch.tensor([[5,3,3]])
criterion(reader_topk_loss, reader_k_loss, retriever_k_scores)

tensor(0.6946)

In [95]:
torch.tensor([[1,2,3],[4,5,6]]).argmax(dim=1)

tensor([2, 2])

### Reader + Frozen Retriever Train

In [7]:
import torch

In [12]:
input_ids = torch.randint(0,255, size=(1,2,512))
attention_mask = torch.randint(0,2, size=(1,2,512))

labels = torch.randint(0,255, size=(1,512))

In [25]:
import torch.nn as nn

In [21]:
reader.tokenizer(" 2 hello world", add_special_tokens=True)

{'input_ids': [204, 21820, 296, 1], 'attention_mask': [1, 1, 1, 1]}

In [24]:
output = reader.model(input_ids=input_ids,attention_mask=attention_mask, labels=labels)

forward 2
init dimension:  torch.Size([1, 2, 512])
2d resize:  torch.Size([1, 1024])
forward 3
2d resize:  torch.Size([1, 1024])
---
candidates flat:  torch.Size([2, 512])
encoder output  torch.Size([2, 512, 768])
candidates concatenation:
last_hidden_state torch.Size([1, 1024, 768])


In [59]:
output.logits.shape

torch.Size([1, 512, 32128])

In [60]:
labels.shape

torch.Size([1, 512])

In [111]:
F.softmax(output.logits, dim=-1)[0][0][111]

tensor(1.2608e-06, grad_fn=<SelectBackward0>)

In [106]:
F.softmax(output.logits.view(512, 32128), dim=-1).gather(1,labels)[0][0]

tensor(1.2608e-06, grad_fn=<SelectBackward0>)

In [137]:
output.loss

tensor(15.1423, grad_fn=<NllLossBackward0>)

In [135]:
outs = torch.mean(-torch.log(F.softmax(output.logits, dim=-1).gather(2,labels.view(1, 512, -1))).view(1, 512), dim=1)

In [136]:
outs

tensor([15.1423], grad_fn=<MeanBackward1>)

In [124]:
F.softmax(output.logits, dim=-1)[0][0][111]

tensor(1.2608e-06, grad_fn=<SelectBackward0>)

In [2]:
reader = FiDReader()

/home/dzigen/miniconda3/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you.

In [ ]:
output_reshape = output[:,:-1].contiguous().view(-1, output.shape[-1])
                trg_batch = trg_batch[:, 1:].contiguous().view(-1)

### Joint Reader + Retriever Train

In [ ]:
def join_evaluate(reader, retriever, loader, criterion,
                  reader_metrics, retriever_metrics, config):
    reader.model.eval()
    retriever.model.eval()
    scores = {
        'reader': {
            'bleu': [],'rouge': [],
            'meteor': [],'em': []
        },
        'retriever': {
            'MRR': [],
            'mAP': []
        }
    }

    losses = []
    process = tqdm(loader)
    for batch in process:
        gc.collect()
        torch.cuda.empty_cache()

        q_ids = batch['q_ids'].to(config.device, non_blocking=True)
        q_masks = batch['q_mask'].to(config.device, non_blocking=True)

        for i in range(len(batch['q_text'])):
            # retriever-part
            texts, scores, metadata = retriever.search(batch['q_text'][i], 
                                             {'input_ids': q_ids[i].view(1,-1),
                                              'attention_mask':q_masks[i].view(1,-1)})

            if 'relevant_d_ids' in batch.keys():
                predicted_d_ids = [meta['in_base_index'] for meta in metadata]

                scores['retriever']['MRR'].append(retriever_metrics.mAP(
                    predicted_d_ids,batch['relevant_d_ids'][i]))
                scores['retriever']['mAP'].append(retriever_metrics.MRR(
                    predicted_d_ids,batch['relevant_d_ids'][i]))

            # reader-part
            prep_txts = list(map(lambda t: "QUESTION: "+t+" PASSAGE: "+batch['q_text'][i], texts))
            tokenized_txts = reader.tokenize(prep_txts)

            output = reader.model(
                input_ids=tokenized_txts['input_ids'].view(1, len(prep_txts), -1), 
                labels=batch['label'][i].view(1,-1), 
                attention_mask=tokenized_txts['attention_mask'].view(1, len(prep_txts), -1))
            
            reader_topk_loss = output.loss.item()

            output = reader.model(
                input_ids=tokenized_txts['input_ids'].view(len(prep_txts), 1, -1), 
                labels=batch['label'][i].view(1,-1), 
                attention_mask=tokenized_txts['attention_mask'].view(len(prep_txts), 1, -1))

            reader_k_loss = ...

            loss = criterion(reader_topk_loss, reader_k_loss, scores)

            losses.append(loss.item())
            process.set_postfix({"avg_loss": np.mean(losses)})

            output = reader.model.generate(
                input_ids=tokenized_txts['input_ids'],
                attention_mask=tokenized_txts['attention_mask'], max_length=128)   

            predicted = reader.tokenizer.batch_decode(output, skip_special_tokens=True)

            scores['reader']['bleu'].append(reader_metrics.bleu(predicted,[batch['label_text'][i]]))
            scores['reader']['rouge'].append(reader_metrics.rouge(predicted,[batch['label_text'][i]]))
            scores['reader']['meteor'].append(reader_metrics.meteor(predicted,[batch['label_text'][i]]))
            scores['reader']['em'].append(reader_metrics.exact_match(predicted,[batch['label_text'][i]]))

def join_train(reader, retriever, train_loader, optimizer, criterion,
                config):
    pass


    # извлекаем k релевантных пассажей по query

    # заного прогоняем пассажи через retriever-часть
    # получаем скоры

    # конкатенируем пассажи и запросы
    # передаём в read модель
    # получем общий loss

    # подаём пассаж для каждлого запроса по отдельности
    # получем loss для каждого пассажа в отдельности
    # делаем stop gradioents

    # комбинируем скор из ретривера, общий лосс от ридера и разделённый лосс от ридера